In [17]:
from __future__ import absolute_import, division, print_function
import numpy as np
from sklearn.utils import shuffle
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from sklearn.preprocessing import minmax_scale 
from tensorflow import feature_column
from tensorflow.keras import layers,Sequential
from tensorflow.keras.layers import DenseFeatures,Dense,Dropout,Conv1D,MaxPooling1D,Flatten
from sklearn.model_selection import train_test_split
import copy
import h5py
print(tf.__version__)   # 2.3

2.6.0


In [18]:
df = pd.read_csv('gandou.csv')
df=df.copy()
print(df.shape)  # (250, 20)
print(df.head(5))
# dataframe =shuffle(df)
print(type(df))

(13611, 17)
    Area  Perimeter  MajorAxisLength  MinorAxisLength  AspectRation  \
0  28395    610.291       208.178117       173.888747      1.197191   
1  28734    638.018       200.524796       182.734419      1.097356   
2  29380    624.110       212.826130       175.931143      1.209713   
3  30008    645.884       210.557999       182.516516      1.153638   
4  30140    620.134       201.847882       190.279279      1.060798   

   Eccentricity  ConvexArea  EquivDiameter    Extent  Solidity  roundness  \
0      0.549812       28715     190.141097  0.763923  0.988856   0.958027   
1      0.411785       29172     191.272751  0.783968  0.984986   0.887034   
2      0.562727       29690     193.410904  0.778113  0.989559   0.947849   
3      0.498616       30724     195.467062  0.782681  0.976696   0.903936   
4      0.333680       30417     195.896503  0.773098  0.990893   0.984877   

   Compactness  ShapeFactor1  ShapeFactor2  ShapeFactor3  ShapeFactor4  Class  
0     0.913358    

In [19]:
labelencoder = LabelEncoder()
df.iloc[:, -1] = labelencoder.fit_transform(df.iloc[:, -1])
df.Class.value_counts()

3    3546
6    2636
5    2027
4    1928
2    1630
0    1322
1     522
Name: Class, dtype: int64

In [20]:
df=shuffle(df)

In [21]:
labels = np.array(df['Class'])
features = df.drop('Class', axis=1)
# features = (features - features.min()) / (features.max() - features.min())
features = (features - features.mean()) / features.std()
features = np.array(features)

In [22]:
features=features[:,:,np.newaxis]


In [26]:
print(type(features))

<class 'numpy.ndarray'>


In [23]:
f_num=[16,32,64,128,256]
k_num=[x for x in range(3, 6)]
m_num=[2,4,6]
dout_num=[0.3,0.5,0.7]
dese_num=[16,32,64,128]
list_loss=[]
list_acc=[]
list_v_loss=[]
list_v_acc=[]
value_num=[]

In [24]:
progress_num=0
for i in range(len(f_num)):
    for j in range(len(k_num)):
        for k in range(len(m_num)):
            for p in range(len(dout_num)):
                for q in range(len(dese_num)):
                    model = Sequential([
                        Conv1D(f_num[i],k_num[j],padding='same',activation='relu',input_shape=(16,1)),#这里一定要有input_shape,16和特征数相对应
                        MaxPooling1D(m_num[k]),
                        Dropout(dout_num[p]),
                        Flatten(),
                        Dense(dese_num[q], activation='relu'),
                        Dense(7, activation='softmax')
            ])
                    print(model.summary())
                    model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',#想0,1,2,3这种就带sparse，像[0,0,1]张量分类就不带sparse
                        metrics=['accuracy'])
                    
                    history=model.fit(features, labels, validation_split=0.2, epochs=50, batch_size=32, shuffle=True)
                    list_loss.append(max(history.history['loss']))
                    list_acc.append(max(history.history['accuracy']))
                    list_v_loss.append(max(history.history['val_loss']))
                    list_v_acc.append(max(history.history['val_accuracy']))
                    ap_value=[]
                    ap_value=[i,j,k,p,q]
                    value_num.append(ap_value)
                    progress_num+=1
                    if progress_num%100==0:
                        print(progress_num)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 16, 16)            64        
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 8, 16)             0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 8, 16)             0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 16)                2064      
_________________________________________________________________
dense_7 (Dense)              (None, 7)                 119       
Total params: 2,247
Trainable params: 2,247
Non-trainable params: 0
____________________________________________________

KeyboardInterrupt: 

In [ ]:

print(max(list_v_acc))



In [ ]:
for i in range(len(list_acc)):
    with open('./rulest/3.txt', 'a+') as f:
        print('acc:'+str(list_acc[i])+',v_acc:'+str(list_v_acc[i])+',num:'+str(value_num[i]),file=f)


In [ ]:
for i in range(16,257):
    for j in range(3,17):
        with open('./rulest/2.txt', 'a+') as f:
            # print('Hello World!', file=f)
            print('f_num:'+str(i)+',k_num:'+str(j)+', acc'+str(list_acc[i+j-19]),file=f)